In [74]:
from sklearn import svm
import pickle
import numpy as np
import os
from sklearn.utils import multiclass as mc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, scale
from lib import accuracy
from lib import labels

In [2]:
data_dir = "../data/"
pickle_file = 'tf_data_mean.pickle'

with open(data_dir + pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = np.matrix(save['train_data'], dtype='float32')
    train_labels = np.matrix(save['train_labels'], dtype='float32')
    valid_dataset = np.matrix(save['validate_data'], dtype='float32')
    valid_labels = np.matrix(save['validate_labels'], dtype='float32')
    test_dataset = np.matrix(save['test_data'], dtype='float32')
    test_bids = list(save['test_business_ids'])
    #test_bids = np.ravel(test_bids)
    del save  # hint to help gc free up memory
    print 'Training set', train_dataset.shape, train_labels.shape
    print 'Validation set', valid_dataset.shape, valid_labels.shape
    print 'Test set', test_dataset.shape, len(test_bids)

# Convert labels to a dict of binarized labels [1. if true, 1. if false]
# So can be used for softmax per label.
#train_labels = labels.binarize_softmax_labels(train_labels)
#valid_labels = labels.binarize_softmax_labels(valid_labels)

Training set (1600, 1024) (1600, 9)
Validation set (400, 1024) (400, 9)
Test set (10000, 1024) 10000


In [9]:
print "What's the percentage of labels?"
for i in range(9):
    print "#"+str(i)+":", np.sum(train_labels[:,i]) / 16
    print "##:", np.sum(valid_labels[:,i]) / 4
    print ""

What's the percentage of labels?
#0: 33.6875
##: 33.0

#1: 49.25
##: 51.25

#2: 50.9375
##: 52.75

#3: 49.875
##: 51.25

#4: 27.625
##: 26.25

#5: 62.125
##: 63.75

#6: 67.875
##: 68.5

#7: 28.6875
##: 28.25

#8: 62.1875
##: 60.75



In [88]:
from sklearn.naive_bayes import BernoulliNB

#LABEL TO PREDICT:

for i in range(9):
    label_to_predict = i
    # predicting 5 based on the other items.
    X = np.delete(train_labels, label_to_predict, 1)
    Y = np.ravel(train_labels[:,label_to_predict])
    valid_X = np.delete(valid_labels, label_to_predict, 1)
    clf = BernoulliNB()
    clf.fit(X, Y)
    predictions = clf.predict(valid_X)
    
    correct_pct = np.sum(np.equal(np.ravel(predictions), np.ravel(valid_labels[:,label_to_predict]))) * 1.0 / len(predictions)
    print "#", label_to_predict, ":",np.sum(np.equal(
        np.ravel(predictions), np.ravel(valid_labels[:,label_to_predict])
    )) * 1.0 / len(predictions)

nb_preds = np.zeros((400, 9))
print ""
for i in range(9):
    label_to_predict = i
    # predicting 5 based on the other items.
    X = np.delete(train_labels, [0, 1, 2, 3, label_to_predict], 1)
    Y = np.ravel(train_labels[:,label_to_predict])
    valid_X = np.delete(valid_labels, [0, 1, 2, 3, label_to_predict], 1)
    clf = BernoulliNB()
    clf.fit(X, Y)
    predictions = clf.predict(valid_X)
    nb_preds[:,i] = predictions
    
    correct_pct = np.sum(np.equal(np.ravel(predictions), np.ravel(valid_labels[:,label_to_predict]))) * 1.0 / len(predictions)
    print "#", label_to_predict, ":",np.sum(np.equal(
        np.ravel(predictions), np.ravel(valid_labels[:,label_to_predict])
    )) * 1.0 / len(predictions)
    
with open(data_dir+"nb_preds.pickle", 'wb') as f:
    # Pickle dictionary using protocol 0.
    pickle.dump(nb_preds, f)


# 0 : 0.6775
# 1 : 0.83
# 2 : 0.8625
# 3 : 0.545
# 4 : 0.815
# 5 : 0.81
# 6 : 0.85
# 7 : 0.8075
# 8 : 0.8

# 0 : 0.695
# 1 : 0.8
# 2 : 0.825
# 3 : 0.5275
# 4 : 0.785
# 5 : 0.79
# 6 : 0.82
# 7 : 0.7825
# 8 : 0.8025


In [65]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(X, Y)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [67]:
predictions.sum() / 4

49.75

In [68]:
np.sum(np.equal(np.ravel(predictions), np.ravel(valid_labels[:,5]))) * 1.0 / len(predictions)
# 84% right (compared to a dismal 56%), but that assumes the inputs are 100% correct.

0.84499999999999997

In [69]:
trainL = labels.matrix_2_labels(train_labels)

trainL = MultiLabelBinarizer().fit_transform(trainL)
#train_labels = matrix_2_labels(train_labels)
trainL = np.asarray(trainL)

#clf = svm.SVC(decision_function_shape='ovr', kernel='linear')


clf = OneVsRestClassifier(svm.LinearSVC(random_state=42)).fit(train_dataset, trainL)
svm_predictions = clf.predict(valid_dataset)

In [70]:
# predicting 5 based on the other items.
X = np.delete(train_labels, label_to_predict, 1)
Y = np.ravel(train_labels[:,label_to_predict])
valid_X = np.delete(svm_predictions, label_to_predict, 1)

print X.shape
print Y.shape
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(X, Y)

(1600, 8)
(1600,)


BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [72]:
predictions = clf.predict(valid_X)
print "old occupancy:", svm_predictions[:,label_to_predict].sum() / 4
print "old accuracy:",np.sum(np.equal(
        np.ravel(svm_predictions[:,label_to_predict]), np.ravel(valid_labels[:,label_to_predict])
    )) * 1.0 / len(svm_predictions[:,label_to_predict])

print "new occupancy:", predictions.sum() / 4
print "new accuracy:",np.sum(np.equal(
        np.ravel(predictions), np.ravel(valid_labels[:,label_to_predict])
    )) * 1.0 / len(predictions)
# Damn, drops down to 54.5% from 62.7%, thus it's worse
# Still,if we can increase the other scores using the neural net, we might be able to have
# enough correct priors to make a better prediction.

old occupancy: 58
old accuracy: 0.6275
new occupancy: 52.25
new accuracy: 0.545


In [73]:
print "old mean_f1_score:", accuracy.mean_f1_score(svm_predictions, valid_labels)

#overwrite the crappy svm predictions and use the bayes prediction
svm_predictions[:,label_to_predict] = predictions
#test new f1_score
print "new mean_f1_score:", accuracy.mean_f1_score(svm_predictions, valid_labels) 

old mean_f1_score: 0.768548180986
new mean_f1_score: 0.752406496281


array([[ 0.,  1.,  1., ...,  1.,  1.,  0.],
       [ 0.,  1.,  1., ...,  1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  1.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  1.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  1.]])

In [ ]:
with open(data_dir + pickle_file, 'rb') as f: